In [1]:
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
import numpy as np
import os
from PIL import Image
import fasttext

images_dir = '../teluguOCR/Dataset/Noised_Images/Image'
Labels_dir = "../teluguOCR/Dataset/strings.txt"
device = torch.device('cpu')

In [2]:
# load the txt file and read the file line by line.
def read_file_lines(filename):
    lines = []
    try:
        with open(filename, 'r') as file:
            for line in file:
                lines.append(line.strip())  # Remove trailing newline characters
    except FileNotFoundError:
        print(f"File '{filename}' not found.")
    except Exception as e:
        print(f"An error occurred: {str(e)}")
    return lines

In [3]:
acchulu = ['అ', 'ఆ', 'ఇ', 'ఈ', 'ఉ', 'ఊ', 'ఋ', 'ౠ', 'ఌ', 'ౡ', 'ఎ', 'ఏ', 'ఐ', 'ఒ', 'ఓ', 'ఔ', 'అం', 'అః']
hallulu = ['క', 'ఖ', 'గ', 'ఘ', 'ఙ',
           'చ', 'ఛ', 'జ', 'ఝ', 'ఞ',
           'ట', 'ఠ', 'డ', 'ఢ', 'ణ',
           'త', 'థ', 'ద', 'ధ', 'న',
           'ప', 'ఫ', 'బ', 'భ', 'మ',
           'య', 'ర', 'ల', 'వ', 'శ', 'ష', 'స', 'హ', 'ళ', 'క్ష', 'ఱ']
vallulu = ['ా', 'ి', 'ీ', 'ు' , 'ూ', 'ృ', 'ౄ', 'ె', 'ే', 'ై', 'ొ', 'ో', 'ౌ', 'ం', 'ః', 'ఁ', 'ౕ', 'ౖ', 'ౢ' ]
connector = ['్']
numbers = ['౦', '౧', '౨', '౩', '౪', '౫', '౬', '౭', '౮', '౯']
splcharacters= [' ', '!', '"', '#', '$', '%', '&', "'", '(', ')',
              '*', '+', ',', '-', '.', '/', ':', ';', '<', '=', '>', '?', '@', '[',
              '\\', ']', '^', '_', '`', '{', '|', '}', '~', '1','2', '3', '4', '5', '6', '7', '8', '9', '0', 'ఽ']
spl = splcharacters + numbers

bases = acchulu + hallulu + spl
vms = vallulu
cms = hallulu

In [4]:
acchulu = ['అ', 'ఆ', 'ఇ', 'ఈ', 'ఉ', 'ఊ', 'ఋ', 'ౠ', 'ఌ', 'ౡ', 'ఎ', 'ఏ', 'ఐ', 'ఒ', 'ఓ', 'ఔ', 'అం', 'అః']
hallulu = ['క', 'ఖ', 'గ', 'ఘ', 'ఙ',
           'చ', 'ఛ', 'జ', 'ఝ', 'ఞ',
           'ట', 'ఠ', 'డ', 'ఢ', 'ణ',
           'త', 'థ', 'ద', 'ధ', 'న',
           'ప', 'ఫ', 'బ', 'భ', 'మ',
           'య', 'ర', 'ల', 'వ', 'శ', 'ష', 'స', 'హ', 'ళ', 'క్ష', 'ఱ']
vallulu = ['ా', 'ి', 'ీ', 'ు' , 'ూ', 'ృ', 'ౄ', 'ె', 'ే', 'ై', 'ొ', 'ో', 'ౌ', 'ం', 'ః', 'ఁ', 'ౕ', 'ౖ', 'ౢ' ]
connector = ['్']
numbers = ['౦', '౧', '౨', '౩', '౪', '౫', '౬', '౭', '౮', '౯']
splcharacters= [' ', '!', '"', '#', '$', '%', '&', "'", '(', ')',
              '*', '+', ',', '-', '.', '/', ':', ';', '<', '=', '>', '?', '@', '[',
              '\\', ']', '^', '_', '`', '{', '|', '}', '~', '1','2', '3', '4', '5', '6', '7', '8', '9', '0', 'ఽ']
spl = splcharacters + numbers

bases = acchulu + hallulu + spl
vms = vallulu
cms = hallulu

print("Bases: ", len(bases))
print("Vms: ", len(vms))
print("Cms: ", len(cms))

characters = bases+vms+cms+connector

base_mapping = {}
i = 2
for x in bases:
  base_mapping[x] = i
  i+=1

vm_mapping = {}
i = 2
for x in vms:
  vm_mapping[x] = i
  i+=1

cm_mapping = {}
i = 2
for x in cms:
  cm_mapping[x] = i
  i+=1

  
# creates a list of ductionaries with each dictionary reporesenting a term
def wordsDicts(s):
  List = []
  for i in range(len(s)):
    x = s[i]
    prev = ''
    if i > 0: prev = s[i-1]
    #----------------------------------is it a base term-----------------------
    if((x in acchulu or x in hallulu)  and prev != connector[0]):
      List.append({})
      List[-1]['base'] = x
    #----------------------------if it is a consonant modifier-----------------
    elif x in hallulu and prev == connector[0]:
      if(len(List) == 0):
        print(x)
      if('cm' not in List[-1]): List[-1]['cm'] = []
      List[len(List)-1]['cm'].append(x)

      #---------------------------if it is a vowel modifier--------------------
    elif x in vallulu:
      if(len(List) == 0):
        print(x)

      if('vm' not in List[-1]): List[-1]['vm'] = []
      List[len(List)-1]['vm'].append(x)

      #----------------------------it is a spl character-----------------------
    elif x in spl:
      List.append({})
      List[len(List)-1]['base'] = x
    else:
      continue
  return List

# def one_hot_encoder(s):
#   List = wordsDicts(s)
#   onehot = []
#   for i in range(len(List)):
#     D = List[i]
#     onehotbase=  [0 for _ in range(len(acchulu) +  len(hallulu) + len(spl))]
#     onehotvm1 =  [0 for _ in range(len(vallulu))]
#     onehotvm2 =  [0 for _ in range(len(vallulu))]
#     onehotvm3 =  [0 for _ in range(len(vallulu))]
#     onehotvm4 =  [0 for _ in range(len(vallulu))]

#     onehotcm1 =  [0 for _ in range(len(hallulu))]   
#     onehotcm2 =  [0 for _ in range(len(hallulu))]   
#     onehotcm3 =  [0 for _ in range(len(hallulu))]   
#     onehotcm4 =  [0 for _ in range(len(hallulu))]   

#     onehotbase[base_mapping[D['base']]-1] = 1

#     it = 1
#     if('vm' in D):
#       for j in D['vm']:
#         if it == 1:
#           onehotvm1[vm_mapping[j]-1] = 1
#         elif it == 2:
#           onehotvm2[vm_mapping[j]-1] = 1
#         elif it == 3:
#           onehotvm3[vm_mapping[j]-1] = 1
#         elif it == 4:
#           onehotvm4[vm_mapping[j]-1] = 1
#         it += 1
    
#     it = 1
#     if('cm' in D):
#       for j in D['cm']:
#         if it == 1:
#           onehotcm1[cm_mapping[j]-1] = 1
#         elif it == 2:
#           onehotcm2[cm_mapping[j]-1] = 1
#         elif it == 3:
#           onehotcm3[cm_mapping[j]-1] = 1
#         elif it == 4:
#           onehotcm4[cm_mapping[j]-1] = 1
#         it += 1


#     onehoti = onehotbase + onehotvm1 + onehotvm2 + onehotvm3 + onehotvm4 + onehotcm1 + onehotcm2 + onehotcm3 + onehotcm4 #size 112 + 4*21 + 4*40 = 356
#     onehot.append(onehoti)
#   encoded = torch.tensor(onehot).float().to(device)
#   return encoded

# def One_Hot_Decoder(List):
#   x = ""
#   for onehoti in List:
#     for i in range(0, 112):
#       if onehoti[i] == 1:
#         x += bases[i]

#     for i in range(196, 236):
#       if onehoti[i] == 1:
#         x += connector[0]
#         x += cms[i-196]
#     for i in range(236, 276):
#       if onehoti[i] == 1:
#         x += connector[0]
#         x += cms[i-236]
#     for i in range(276, 316):
#       if onehoti[i] == 1:
#         x += connector[0]
#         x += cms[i-276]
#     for i in range(316, 356):
#       if onehoti[i] == 1:
#         x += connector[0]
#         x += cms[i-316]

#     for i in range(112, 133):
#       if onehoti[i] == 1:
#         x += vms[i-112]
#     for i in range(133, 154):
#       if onehoti[i] == 1:
#         x += vms[i-133]
#     for i in range(154, 175):
#       if onehoti[i] == 1:
#         x += vms[i-154]
#     for i in range(175, 196):
#       if onehoti[i] == 1:
#         x += vms[i-175]
#   return 

Bases:  108
Vms:  19
Cms:  36


In [5]:
def index_encoding(s):
  List = wordsDicts(s)
  onehot = []
  for i in range(len(List)):
    D = List[i]
    onehotbase=  [1]
    onehotvm1 =  [1]
    onehotvm2 =  [1]
    onehotvm3 =  [1]
    onehotvm4 =  [1]
    onehotcm1 =  [1]
    onehotcm2 =  [1]
    onehotcm3 =  [1]
    onehotcm4 =  [1]


    onehotbase[0] = base_mapping[D['base']]

    it = 1
    if('vm' in D):
      for j in D['vm']:
        if it == 1:
          onehotvm1[0] = vm_mapping[j]
        elif it == 2:
          onehotvm2[0] = vm_mapping[j]
        elif it == 3:
          onehotvm3[0] = vm_mapping[j]
        elif it == 4:
          onehotvm4[0] = vm_mapping[j]
        it += 1
    
    it = 1
    if('cm' in D):
      for j in D['cm']:
        if it == 1:
          onehotcm1[0] = cm_mapping[j]
        elif it == 2:
          onehotcm2[0] = cm_mapping[j]
        elif it == 3:
          onehotcm3[0] = cm_mapping[j]
        elif it == 4:
          onehotcm4[0] = cm_mapping[j]
        it += 1
    onehoti = onehotbase + onehotvm1 + onehotvm2 + onehotvm3 + onehotvm4 + onehotcm1 + onehotcm2 + onehotcm3 + onehotcm4 #size 110 + 4*21 + 4*38 = 346
    onehot.append(onehoti)
  return onehot

def index_decoder(List):
  x = ""
  for onehoti in List:
    if onehoti[0] > 1:
      x += bases[onehoti[0]-2]

    if onehoti[5] > 1:
      x += connector[0]
      x += cms[onehoti[5]-2]
    if onehoti[6] > 1:
      x += connector[0]
      x += cms[onehoti[6]-2]
    if onehoti[7] > 1:
      x += connector[0]
      x += cms[onehoti[7]-2]
    if onehoti[8] > 1:
      x += connector[0]
      x += cms[onehoti[8]-2]

    if onehoti[1] > 1:
      x += vms[onehoti[1]-2]
    if onehoti[2] > 1:
      x += vms[onehoti[2]-2]
    if onehoti[3] > 1:
      x += vms[onehoti[3]-2]
    if onehoti[4] > 1:
      x += vms[onehoti[4]-2]
  return x

In [6]:
s = "అందరికి శుభాకాంక్షలు"
encoded = index_encoding(s)
print(len(encoded))
print(len(encoded[0]))
index_decoder(encoded)

10
9


'అందరికి శుభాకాంక్షలు'

In [7]:
lengths = []

In [10]:
lines = read_file_lines("/home/ocr/teluguOCR/Dataset/final_strings.txt")

a = b = 0

index = 1
for i in range(1, 189531 + 1):
    image = torch.load("/home/ocr/teluguOCR/Dataset/Noised_Images/Image" + str(i) + ".pt")
    line = lines[i-1]
    if(image.shape[1] > 800):
        continue
    if line[0] not in bases:
        continue
    label = index_encoding(line)
    label = torch.tensor(label)
    image = 255 - image

    a, b = max(a, image.shape[1]), max(b, label.shape[0])

    torch.save(image, "/home/ocr/teluguOCR/Dataset/Cropped_Dataset/Images/Image" + str(index) + ".pt")
    torch.save(label, "/home/ocr/teluguOCR/Dataset/Cropped_Dataset/Labels/Label" + str(index) + ".pt")
    print(index, end = '\r')
    index += 1

torch.Size([3, 40, 296])


NameError: name 'jhvbhvb' is not defined

In [9]:
print(a, " | ", b)

800  |  158


In [ ]:
# import matplotlib.pyplot as plt

# LIST_Ten = torch.tensor(LIST)

# print(LIST_Ten.shape)

# # Plotting the distribution of lengths
# plt.hist(LIST_Ten[:, 1], bins=100)
# plt.show()